Yahoo Finance request

In [ ]:
%pip install yfinance

In [ ]:
import yfinance as yf
fx = yf.Ticker("EURUSD=X")
fx.info

In [ ]:
fxHist = fx.history(start="2023-01-10", end="2023-01-11", interval= "1h")
fxHist

In [ ]:
fxDict = fxHist.to_json(orient='index') ##convertendo resultado em json
print(fxDict)

Conectando com cosmoDB-MS

In [ ]:
%pip install azure-cosmos

In [12]:
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import datetime
import config